# Visualizing Water Quality of the Taihu Lake

## Introduction

Eutrophication, the excessive enrichment of water bodies with nutrients, particularly nitrogen and phosphorus, leads to the accelerated growth of algae and other aquatic plants. As these organisms die and decompose, oxygen is depleted, harming aquatic life and disrupting the ecological balance. It is crucial to monitor eutrophication as it directly impacts water quality, biodiversity, and the overall health of aquatic ecosystems. Algal blooms and oxygen depletion can have far-reaching consequences, including reduced water quality for drinking, fishing, and recreational activities.

Our region of interest, Lake Taihu in China has faced severe eutrophication issues, primarily due to industrial and domestic wastewater, agricultural runoff, and untreated sewage discharge. These pollutants fuel the growth of algae, notably cyanobacteria, leading to harmful algal blooms. The resulting toxins pose risks to both aquatic life and human health, emphasizing the urgency of effective monitoring and management of the lake's water quality.

### Import necessary libraries

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import getpass

from ipyleaflet import GeoJSON, Map, basemaps
import matplotlib.pyplot as plt

from sentinelhub import (
    SHConfig,
    CRS,
    BBox,
    DataCollection,
    MimeType,
    MosaickingOrder,
    SentinelHubRequest,
)

# The following is not a package. It is a file utils.py which should be in the same folder as this notebook.
from utils import plot_image

### Credentials

Credentials for Sentinel Hub services (`client_id` & `client_secret`) can be obtained by navigating to your [Dashboard](https://shapps.dataspace.copernicus.eu/dashboard/#/). In the User Settings you can create a new OAuth Client to generate these credentials. For more detailed instructions, visit the relevent [documentation page](https://documentation.dataspace.copernicus.eu/APIs/SentinelHub/Overview/Authentication.html).

Now that you have your `client_id` & `client_secret`, it is recommended to configure a new profile in your Sentinel Hub Python package. Instructions on how to configure your Sentinel Hub Python package can be found [here](https://sentinelhub-py.readthedocs.io/en/latest/configure.html). Using these instructions you can create a profile specific to using the package for accessing Copernicus Data Space Ecosystem data collections. This is useful as changes to the the config class are usually only temporary in your notebook and by saving the configuration to your profile you won't need to generate new credentials or overwrite/change the default profile each time you rerun or write a new Jupyter Notebook. 

If you are a first time user of the Sentinel Hub Python package for Copernicus Data Space Ecosystem, you should create a profile specific to the Copernicus Data Space Ecosystem. You can do this in the following cell:

In [ ]:
# Only run this cell if you have not created a configuration.

config = SHConfig()
config.sh_client_id = getpass.getpass("Enter your SentinelHub client id")
config.sh_client_secret = getpass.getpass("Enter your SentinelHub client secret")
config.sh_token_url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
config.sh_base_url = "https://sh.dataspace.copernicus.eu"
# config.save("cdse")

However, if you have already configured a profile in Sentinel Hub Python for the Copernicus Data Space Ecosystem, then you can uncomment and run the below cell by entering the profile name as a string replacing `profile_name` (which is `cdse` in our case, feel free to change this when you save your profile).

In [ ]:
#config = SHConfig("profile_name")

## Process API
The [Process API](https://documentation.dataspace.copernicus.eu/APIs/SentinelHub/Process.html) is an essential element in the Copernicus Data Space Ecosystem. It allows the generation of personalized visual representations of satellite data and enables users to process and analyze data in the cloud.


### Example: Insert the Ulyssys Water Quality Viewer script to visualize the water quality of our area of interest.

We build the request according to the [API Reference](https://documentation.dataspace.copernicus.eu/APIs/SentinelHub/ApiReference.html), using the `SentinelHubRequest` class. Each Process API request also needs an [evalscript](https://documentation.dataspace.copernicus.eu/APIs/SentinelHub/Evalscript/V3.html).


The information that we specify in the `SentinelHubRequest` object is: 

 * an evalscript,
 * a list of input data collections with time interval,
 * a format of the response,
 * a bounding box and it's size (size or resolution).

 First, let us define the area of interest.


In [ ]:
taihu_lake_bbox = BBox((119.7301,30.919636,120.684429,31.661304), crs=CRS.WGS84).transform(3035)            # Convert to 3035 to get crs with meters as units
time_interval = "2023-09-01", "2023-09-30"

resolution = (100,100)

x, y = taihu_lake_bbox.transform(4326).middle

# Add OSM background
overview_map = Map(basemap=basemaps.OpenStreetMap.Mapnik, center=(y, x), zoom=8)

# Add geojson data
geo_json = GeoJSON(data=taihu_lake_bbox.transform(4326).geojson)
overview_map.add_layer(geo_json)

# Display
overview_map

Next, we define the evalscript (or the Custom Script). To observe the water quality, we have chosen the [Ulyssys Water Quality Viewer Script](https://custom-scripts.sentinel-hub.com/custom-scripts/sentinel-2/ulyssys_water_quality_viewer/). This script helps in monitoring eutrophication in Lake Taihu, and other water bodies, showcasing various water quality parameters, including nutrient concentrations and algal bloom occurrences. This analysis helps stakeholders and researchers to identify patterns and take timely action to mitigate the eutrophication problem. 

#### Description of the script

Ulyssys Water Quality Viewer (UWQV) is a custom script to dynamically visualize the chlorophyll and sediment conditions of water bodies on both Sentinel-2 and Sentinel-3 images.

The visualization you see is a product of two masking operations and two water quality parameter visualizations:

- cloud masking
- water masking

and,

- suspended sediment concentration visualization
- chlorophyll concentration visualization

Detection of water quality parameters is based on either simple band intensity for suspended sediment or the reflectance line height (RLH) principle for chlorophyll. Calculating RLH involves selecting the a central band, and calculating the difference ("height") of reflectance of that band compared to a baseline calculated by linear interpolation between the values of the neighbouring two bands. By choosing a band close to a chlorophyll fluorescence peak this simple model essentially answers the question "how much additional light is received from chlorophyll flourescence compared to the background reflectance?"

For Sentinel-2, the band 05 at 705 is used as a representation of chlorophyll flourescence and the baseline is calculated from B04 and B06.
The calculation is a linear interpolation of an estimated value of B05 in absence of chlorophyll, essentially a weighted average of B04 and B06, inversely weighted by the distance of the wavelengths of these bands from the wavelength of B05.

Find more technical details about the script [here](https://custom-scripts.sentinel-hub.com/custom-scripts/sentinel-2/ulyssys_water_quality_viewer/doc/Ulyssys_Water_Quality_Viewer_supplementary_preprint.pdf).

You can also find various custom evalscripts which might match your application in our [Custom Scripts Repository](https://custom-scripts.sentinel-hub.com/custom-scripts/).

In [ ]:
custom_script = """
//VERSION=3
//VERSION=3
function setup() {
  return {
    input: [{
      bands: [
        "B02",
        "B03",
        "B04",
        "B05",
        "B06",
        "B07",
        "B08",
        "B8A",
        "B09",
        "B11",
      ]
    }],
    output: [
      { id: "default", bands: 3 },
      { id: "chlorophyllIndex", bands: 1, sampleType: "FLOAT32" },
      { id: "sedimentIndex", bands: 1, sampleType: "FLOAT32" },
    ]
  }
}

const PARAMS = {
  // Indices
  chlIndex: 'default',
  tssIndex: 'default',
  watermaskIndices: ['ndwi', 'hol'],
  // Limits
  chlMin: -0.005,
  chlMax: 0.05,
  tssMin: 0.075,
  tssMax: 0.185,
  waterMax: 0,
  cloudMax: 0.02,
  // Graphics
  foreground: 'default',
  foregroundOpacity: 1,
  background: 'default',
  backgroundOpacity: 1.0
};
//* PARAMS END

/**
 * Returns indices object used for output calculation
 * The returned object is different for Sentinel-2 and Sentinel-3 satellites
  * Here only defined as strings and gets evaluated only when really needed
 * (Tip 4: Calculate as needed at https://medium.com/sentinel-hub/custom-scripts-faster-cheaper-better-83f73894658a)
 * natural: natural (rgb) color image
 * chl: chlorophyll indices
 * tss: sediment indices
 * watermask: watermask indices *
 *
 * @param {boolean} isSentinel3: is it Sentinel-3 or not (=Sentinel-2)
 */
function getIndices(sample) {
  return {
      natural: [2.5*sample.B04,2.5*sample.B03,2.5*sample.B02],
      chl: {
        rlh: sample.B05-sample.B04-(sample.B07-sample.B04*((0.705-0.665)*1000.0))/((0.783-0.665)*1000.0),
        mci: sample.B05-((0.74-0.705)/(0.74-0.665))*sample.B04-(1.0-(0.74-0.705)/(0.74-0.665))*sample.B06
      },
      tss: {
        b05: sample.B05
      },
      watermask: {
        ndwi: (sample.B03-sample.B08)/(sample.B03+sample.B08)
      }
    };
}

/**
 * Blends between two layers
 * Uses https://pierre-markuse.net/2019/03/26/sentinel-3-data-visualization-in-eo-browser-using-a-custom-script/
 *
 * @param {Object} layer1: first (top) layer
 * @param {Object} layer2: second (bottom) layer
 * @param {number} opacity1: first layer opacity
 * @param {number} opacity2: second layer opacity
 */
function blend(layer1, layer2, opacity1, opacity2) {
  return layer1.map(function (num, index) {
    return (num / 100) * opacity1 + (layer2[index] / 100) * opacity2;
  });
}

/**
 * Returns an opacity (alpha) value between 0 and 100 for a given index based on min and max values
 *
 * @param {Object} index: selected index
 * @param {number} min: user defined minimum value
 * @param {number} max: user defined maximum value
 */
function getAlpha(index, min, max) {
  if (min + (max - min) / 2 < index) {
    return 100;
  }
  return index <= min ?
    0 :
    index >= max ?
      1 :
      100 * ((index - min / 2) / (max - min));
}

/**
 * Returns a color palette for chlorophyll or sediment index
 *
 * @param {String} type: palette type ('chl' for chlorophyll, 'tss' for sediment)
 * @param {Object} index: user selected index
 * @param {number} min: user defined minimum value
 * @param {number} max: user defined maximum value
 * @param {boolean} isSentinel3Flh: is it Sentinel3 && is 'flh' is the user selected chlorophyll index (only for 'chl' type)
 */
function getColors(type, index, min, max) {
  let colors, palette;
  switch (type) {
    case 'chl':
      palette = [
        [0.0034, 0.0142, 0.163], // #01042A (almost black blue)
        [0, 0.416, 0.306], // #006A4E (bangladesh green)
        [0.486, 0.98, 0], //#7CFA00 (dark saturated chartreuse)
        [0.9465, 0.8431, 0.1048], //#F1D71B (light washed yellow)
        [1, 0, 0] // #FF0000 (red)
      ];
      
      colors = colorBlend(
        index,
        [min, min + (max - min) / 3, (min + max) / 2, max - (max - min) / 3, max],
        palette
      );
      break;
    case 'tss':
      palette = [
        [0.961, 0.871, 0.702], // #F5DEB3 (wheat)
        [0.396, 0.263, 0.129] // #654321 (dark brown)
      ];
      colors = colorBlend(
        index,
        [min, max],
        palette
      );
      break;
    default:
      break;
  }
  return colors;
}

/**
 * Returns true if the pixel covers area of pure water without any cloud, shadow or snow, otherwise returns false
 * Based on the algorithm by Hollstein et al. at https://custom-scripts.sentinel-hub.com/custom-scripts/sentinel-2/hollstein/
 *
 * @param {boolean} isSentinel3: is it Sentinel-3 or not (=Sentinel-2)
 */
function isPureWater(sample) {
  return sample.B03 < 0.319 && sample.B8A < 0.166 && sample.B03 - sample.B07 >= 0.027 && sample.B09 - sample.B11 < 0.021;
}

/**
 * Returns whether the pixel is marked as cloud
 * Based on the algorithm by the Braaten-Cohen-Yang cloud detector at https://github.com/sentinel-hub/custom-scripts/tree/master/sentinel-2/cby_cloud_detection
 *
 * @param {number} limit: user defined cloud limit
 * @param {boolean} isSentinel3: is it Sentinel-3 or not (=Sentinel-2)
 */
function isCloud(sample,limit) {
  const bRatio = (sample.B02 - 0.175) / (0.39 - 0.175);
  return bRatio > 1 || (bRatio > 0 && (sample.B04 - sample.B06) / (sample.B04 + sample.B06) > limit);
}

/**
 * Returns an evaluated code of a string
 * This was needed because functions with eval() won't make it through minification
 *
 * @param {String} s: input string to evaluate
 */
function getEval(s,sample) {
  return eval(s);
}

/**
 * Returns whether the pixel is marked as water (not land, cloud or snow) based on the array of indices given by the user
 *
 * @param {Object} params: user defined parameters
 * @param {Array<String>} indices: array of water indices given by the user. Possible values: "ndwi", "hol", "bcy" and any of their combinations.
 * @param {boolean} isSentinel3: is it Sentinel-3 or not (=Sentinel-2)
 */
function isWater(sample,availableWatermaskIndices, selectedWatermaskIndices, waterMax, cloudMax) {
  if (selectedWatermaskIndices.length === 0) {
    return true;
  } else {
    let isItWater = true;
    for (let i = 0; i < selectedWatermaskIndices.length; i++) {
      const wm = selectedWatermaskIndices[i];
      if (wm == "ndwi" && getEval(availableWatermaskIndices.ndwi) < waterMax) {
        isItWater = false;
        break;
      } else if (wm == "hol" && !isPureWater(sample)) {
        isItWater = false;
        break;
      } else if (wm == "bcy" && isCloud(sample,cloudMax)) {
        isItWater = false;
        break;
      }
    }
    return isItWater;
  }
}

/**
 * Returns background layer
 *
 * @param {String | Array<number>} background: predefined or custom background color
 * @param {Array<numer>} naturalIndex: natural color index
 * @param {number} opacity: background opacity from 0 to 1 (floating value)
 */
function getBackground(background, naturalIndex, opacity) {
  let backgroundLayer;
  let isRgb = false;
  const alpha = parseInt(opacity * 100);
  // Default should be the natural layer
  if (background === 'default' || background === 'natural') {
    backgroundLayer = getEval(naturalIndex);
    isRgb = true;
  } else if (background === 'black') {
    // Black background
    backgroundLayer = [0, 0, 0];
  } else if (background === 'white') {
    // White background
    backgroundLayer = [1, 1, 1];
  } else {
    // Custom rgb colors array (eg. [255, 255, 0])
    backgroundLayer = getStaticColor(background);
  }
  // Only calculate alpha is really needed
  return isRgb || opacity === 1 ? backgroundLayer : blend(backgroundLayer, getEval(naturalIndex), alpha, 100 - alpha);
}

/**
 * Returns foreground layer
 *
 * @param {String | Array<number>} foreground: predefined or custom foreground color
 * @param {*} backgroundLayer: background layer (for blending)
 * @param {*} naturalIndex: natural layer
 * @param {*} opacity: foreground opacity from 0 to 1 (floating value)
 */
function getForeground(foreground, backgroundLayer, naturalIndex, opacity) {
  let layer;
  const alpha = parseInt(opacity * 100);
  if (foreground === 'natural') {
    layer = getEval(naturalIndex);
  } else {
    layer = getStaticColor(foreground);
  }
  return opacity === 1 ? layer : blend(layer, backgroundLayer, alpha, 100 - alpha);
}

/**
 * Transforms RGB 0-255 colors to 0.0-1.0
 *
 * @param {[number, number, number]} colorArray: 3-element array of RGB colors (0-255)
 */
function getStaticColor(colorArray) {
  return [colorArray[0] / 255, colorArray[1] / 255, colorArray[2] / 255];
}

/**
 * Runs the main calculation and returns the value for each pixel
 *
 * @param {Object} params: user defined parameters
 */
function getValue(params,sample) {
  let chlIndex, chlLayer, tssIndex, tssLayer, tssAlpha;
  const chl = params.chlIndex;
  const tss = params.tssIndex;
  const background = params.background;
  const foreground = params.foreground;
  const foregroundOpacity = params.foregroundOpacity;
  // Get the indices that could potentially be used
  const indices = getIndices(sample);
  // Define background layer
  const backgroundLayer = getBackground(background, indices.natural, params.backgroundOpacity);
  // Decide whether the pixel can be assumed as water
  // Return background layer if it is not water
  if (!isWater(indices.watermask, params.watermaskIndices, params.waterMax, params.cloudMax)) {
    return backgroundLayer;
  }
  // Return a static color if set so with opacity
  if (foreground !== 'default') {
    return getForeground(foreground, backgroundLayer, indices.natural, foregroundOpacity);
    
  }
  let value;
  // Define the chlorophyll layer if needed
  if (chl !== null) {
    // In case of 'default' set proper algorighm
    const alg = chl === 'default' ?  'mci' : chl;
    chlIndex = getEval(indices.chl[alg],sample);
    chlLayer = getColors('chl', chlIndex, params.chlMin, params.chlMax, (alg === 'flh'));
  }
  // Define the sediment layer if needed
  if (tss !== null) {
    // In case of 'default' set proper algorighm
    const alg = tss === 'default' ? ('b05') : tss;
    tssIndex = getEval(indices.tss[alg],sample);
    tssLayer = getColors('tss', tssIndex, params.tssMin, params.tssMax);
    tssAlpha = getAlpha(tssIndex, params.tssMin, params.tssMax);
  }
  // Calculate output value
  if (chl !== null && tss !== null) {
    // Blend layers if both chlorophyll and sediment layers are requested
    // Put sediment layer on top of chlorophyll layer with alpha
    value = blend(tssLayer, chlLayer, tssAlpha, 100 - tssAlpha);
  } else if (chl !== null && tss === null) {
    // Chlorophyll layer only if sediment layer is null
    value = chlLayer;
  } else if (tss !== null && chl === null) {
    // Sediment layer only if chlorophyll layer is null
    // Put sediment layer on top of natural layer with alpha
    value = blend(tssLayer, backgroundLayer, tssAlpha, 100 - tssAlpha);
  } else {
    // Natural color layer if both chlorophyll and sediment layers are null (which does not make much sense)
    value = backgroundLayer;
  }
  // Return foreground (with opacity if needed on top of background)
  const foregroundAlpha = parseInt(foregroundOpacity * 100);

  return {
    rgb_value: foregroundOpacity === 1 ? value : blend(value, backgroundLayer, foregroundAlpha, 100 - foregroundAlpha),
    chl_index: chlIndex,
    sed_index: tssIndex,
  };

}

function evaluatePixel(samples) {
  let value = getValue(PARAMS,samples);
   return {
    default: value.rgb_value,
    chlorophyllIndex: [value.chl_index],
    sedimentIndex: [value.sed_index],
  };
}

"""

In [ ]:
time_interval = "2023-09-01", "2023-09-30"
request_UWQV = SentinelHubRequest(
    evalscript=custom_script,
    input_data=[
        SentinelHubRequest.input_data(
            data_collection=DataCollection.SENTINEL2_L2A.define_from(
                "s2l2a", service_url=config.sh_base_url
            ),
            time_interval=time_interval,
            mosaicking_order=MosaickingOrder.LEAST_CC,
        )
    ],
    responses=[SentinelHubRequest.output_response("default", MimeType.TIFF),
               SentinelHubRequest.output_response("chlorophyllIndex", MimeType.TIFF),
               SentinelHubRequest.output_response("sedimentIndex", MimeType.TIFF)
               ],
    bbox=taihu_lake_bbox,
    resolution=resolution,
    config=config,
    data_folder="./data"
)

Let us save the requested data and view the visualization.

In [ ]:
download_data = request_UWQV.get_data(save_data=True)[0]
download_data.keys()

In [ ]:
img = download_data["default.tif"]
plot_image(img, factor=1/255, clip_range=(0, 1))

You can also take a look at the indices and do further processing using this data by extracting them as shown:

In [ ]:
chlorophyll_index = download_data['chlorophyllIndex.tif']
sediment_index = download_data['sedimentIndex.tif']

f, axarr = plt.subplots(1,2,figsize=(12, 12))
axarr[0].imshow(chlorophyll_index)
axarr[1].imshow(sediment_index)

### Summary

This was a short tutorial on how to use a custom script in the Jupyter Notebook to begin your analysis. From here, you can study the pixels, derive statistics and create a workflow that is suiltable for your problem statement. You can find more information about the different APIs and various examples in the [documentation](https://documentation.dataspace.copernicus.eu/APIs/SentinelHub.html). Check out the [Custom Scripts](https://custom-scripts.sentinel-hub.com/custom-scripts/) for more examples. 